# Modelo ARIMA (AutoRegressive Integrated Moving Average)

Este notebook implementa modelos ARIMA para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window

**Configuración de modelos:**
- Producto 1: ARIMA(0, 2, 0) con optimización bayesiana
- Producto 2: ARIMA(2, 1, 9) con optimización bayesiana

Posteriormente, se utiliza Optimización Bayesiana para encontrar los mejores hiperparámetros y entrenar el modelo final para producción.


In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.tsa.stattools import adfuller
from scipy import stats
import warnings
import optuna
warnings.filterwarnings('ignore')


In [2]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función ARIMA

Implementación del modelo ARIMA que permite especificar los órdenes (p, d, q):
- **p**: Orden del componente autorregresivo (AR)
- **d**: Orden de diferenciación
- **q**: Orden del componente de media móvil (MA)


In [3]:
def arima_forecast(data, order, forecast_horizon=1):
    """
    Calcula el modelo ARIMA y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    order : tuple
        Orden del modelo ARIMA (p, d, q)
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float or array
        Predicción(es) usando ARIMA
    """
    if len(data) < 2:
        return data[-1] if len(data) > 0 else 0
    
    # Convertir a pandas Series
    data_series = pd.Series(data)
    
    try:
        # Crear y ajustar el modelo ARIMA
        model = ARIMA(data_series, order=order)
        fit = model.fit()
        
        # Realizar predicción
        forecast = fit.forecast(steps=forecast_horizon)
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            return float(forecast.iloc[0]) if hasattr(forecast, 'iloc') else float(forecast[0])
        else:
            # Devolver array de predicciones
            if hasattr(forecast, 'values'):
                return forecast.values
            else:
                return np.array(forecast)
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        print(f"Advertencia en ARIMA: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [4]:
def walk_forward_validation(dataset, order, train_size_min=24):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    order : tuple
        Orden del modelo ARIMA (p, d, q)
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = arima_forecast(
                    train_data, 
                    order=order,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se utiliza una ventana fija de tamaño constante que se desliza a lo largo de la serie temporal.


In [5]:
def rolling_window_validation(dataset, order, train_window_size=24):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    order : tuple
        Orden del modelo ARIMA (p, d, q)
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = arima_forecast(
                    train_data, 
                    order=order,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [6]:
def expanding_window_validation(dataset, order, train_size_min=24):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    order : tuple
        Orden del modelo ARIMA (p, d, q)
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento (por defecto 24)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = arima_forecast(
                    train_data, 
                    order=order,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.

**Configuración de modelos según análisis exploratorio:**
- Producto 1: ARIMA(0, 2, 0)
- Producto 2: ARIMA(2, 1, 9)


In [7]:
# Parámetros de validación
TRAIN_SIZE_MIN = 24  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 24  # Tamaño fijo para Rolling Window

# Órdenes ARIMA según análisis exploratorio
ORDER_PRODUCTO1 = (0, 2, 0)  # ARIMA(0, 2, 0)
ORDER_PRODUCTO2 = (2, 1, 9)  # ARIMA(2, 1, 9)

print("=" * 60)
print("CONFIGURACIÓN DE MODELOS ARIMA")
print("=" * 60)
print(f"Producto 1: ARIMA{ORDER_PRODUCTO1}")
print(f"Producto 2: ARIMA{ORDER_PRODUCTO2}")
print(f"Tamaño mínimo de entrenamiento: {TRAIN_SIZE_MIN}")
print(f"Tamaño de ventana fija (Rolling): {TRAIN_WINDOW_SIZE}")


CONFIGURACIÓN DE MODELOS ARIMA
Producto 1: ARIMA(0, 2, 0)
Producto 2: ARIMA(2, 1, 9)
Tamaño mínimo de entrenamiento: 24
Tamaño de ventana fija (Rolling): 24


In [8]:
# Ejecutar Walk-Forward Validation para cada producto
print("\n" + "=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)

results_wf = {}

# Producto 1 con ARIMA(0, 2, 0)
print("\n--- Producto 1 con ARIMA(0, 2, 0) ---")
results_wf['producto1'] = walk_forward_validation(
    df[['producto1']], 
    order=ORDER_PRODUCTO1,
    train_size_min=TRAIN_SIZE_MIN
)['producto1']

# Producto 2 con ARIMA(2, 1, 9)
print("\n--- Producto 2 con ARIMA(2, 1, 9) ---")
results_wf['producto2'] = walk_forward_validation(
    df[['producto2']], 
    order=ORDER_PRODUCTO2,
    train_size_min=TRAIN_SIZE_MIN
)['producto2']



WALK-FORWARD VALIDATION

--- Producto 1 con ARIMA(0, 2, 0) ---

=== Walk-Forward Validation para producto1 ===
  Iteración 20/103: RMSE=4.7466, MAE=4.7466
  Iteración 40/103: RMSE=0.1837, MAE=0.1837
  Iteración 60/103: RMSE=19.8744, MAE=19.8744
  Iteración 80/103: RMSE=14.7706, MAE=14.7706
  Iteración 100/103: RMSE=13.7142, MAE=13.7142
  Total de iteraciones: 103

--- Producto 2 con ARIMA(2, 1, 9) ---

=== Walk-Forward Validation para producto2 ===
  Iteración 20/103: RMSE=4.0520, MAE=4.0520
  Iteración 40/103: RMSE=10.9296, MAE=10.9296
  Iteración 60/103: RMSE=7.0046, MAE=7.0046
  Iteración 80/103: RMSE=13.3613, MAE=13.3613
  Iteración 100/103: RMSE=0.3407, MAE=0.3407
  Total de iteraciones: 103


In [9]:
# Ejecutar Rolling Window Validation para cada producto
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)

results_rw = {}

# Producto 1 con ARIMA(0, 2, 0)
print("\n--- Producto 1 con ARIMA(0, 2, 0) ---")
results_rw['producto1'] = rolling_window_validation(
    df[['producto1']], 
    order=ORDER_PRODUCTO1,
    train_window_size=TRAIN_WINDOW_SIZE
)['producto1']

# Producto 2 con ARIMA(2, 1, 9)
print("\n--- Producto 2 con ARIMA(2, 1, 9) ---")
results_rw['producto2'] = rolling_window_validation(
    df[['producto2']], 
    order=ORDER_PRODUCTO2,
    train_window_size=TRAIN_WINDOW_SIZE
)['producto2']



ROLLING WINDOW VALIDATION

--- Producto 1 con ARIMA(0, 2, 0) ---

=== Rolling Window Validation para producto1 ===
  Iteración 20/103: RMSE=4.7466, MAE=4.7466
  Iteración 40/103: RMSE=0.1837, MAE=0.1837
  Iteración 60/103: RMSE=19.8744, MAE=19.8744
  Iteración 80/103: RMSE=14.7706, MAE=14.7706
  Iteración 100/103: RMSE=13.7142, MAE=13.7142
  Total de iteraciones: 103

--- Producto 2 con ARIMA(2, 1, 9) ---

=== Rolling Window Validation para producto2 ===
  Iteración 20/103: RMSE=8.5845, MAE=8.5845
  Iteración 40/103: RMSE=14.9208, MAE=14.9208
  Iteración 60/103: RMSE=7.4121, MAE=7.4121
  Iteración 80/103: RMSE=0.3477, MAE=0.3477
  Iteración 100/103: RMSE=10.3554, MAE=10.3554
  Total de iteraciones: 103


In [10]:
# Ejecutar Expanding Window Validation para cada producto
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)

results_ew = {}

# Producto 1 con ARIMA(0, 2, 0)
print("\n--- Producto 1 con ARIMA(0, 2, 0) ---")
results_ew['producto1'] = expanding_window_validation(
    df[['producto1']], 
    order=ORDER_PRODUCTO1,
    train_size_min=TRAIN_SIZE_MIN
)['producto1']

# Producto 2 con ARIMA(2, 1, 9)
print("\n--- Producto 2 con ARIMA(2, 1, 9) ---")
results_ew['producto2'] = expanding_window_validation(
    df[['producto2']], 
    order=ORDER_PRODUCTO2,
    train_size_min=TRAIN_SIZE_MIN
)['producto2']



EXPANDING WINDOW VALIDATION

--- Producto 1 con ARIMA(0, 2, 0) ---

=== Expanding Window Validation para producto1 ===
  Iteración 20/103: RMSE=4.7466, MAE=4.7466
  Iteración 40/103: RMSE=0.1837, MAE=0.1837
  Iteración 60/103: RMSE=19.8744, MAE=19.8744
  Iteración 80/103: RMSE=14.7706, MAE=14.7706
  Iteración 100/103: RMSE=13.7142, MAE=13.7142
  Total de iteraciones: 103

--- Producto 2 con ARIMA(2, 1, 9) ---

=== Expanding Window Validation para producto2 ===
  Iteración 20/103: RMSE=4.0520, MAE=4.0520
  Iteración 40/103: RMSE=10.9296, MAE=10.9296
  Iteración 60/103: RMSE=7.0046, MAE=7.0046
  Iteración 80/103: RMSE=13.3613, MAE=13.3613
  Iteración 100/103: RMSE=0.3407, MAE=0.3407
  Total de iteraciones: 103


In [11]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio: {rmse_promedio:.4f}")
        print(f"  MAE Promedio: {mae_promedio:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio: 10.5461
  MAE Promedio: 10.5461
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 9.6780
  MAE Promedio: 9.6780
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio: 10.5461
  MAE Promedio: 10.5461
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 10.6872
  MAE Promedio: 10.6872
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - EXPANDING WINDOW

producto1:
  RMSE Promedio: 10.5461
  MAE Promedio: 10.5461
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 9.6780
  MAE Promedio: 9.6780
  Número de iteraciones: 103


## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [12]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
mejor_validacion = {}
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    mejor_validacion[producto] = mejor['Validacion']
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  MAE_Promedio
producto1     Walk-Forward      10.546098     10.546098
producto1   Rolling Window      10.546098     10.546098
producto1 Expanding Window      10.546098     10.546098
producto2     Walk-Forward       9.677990      9.677990
producto2   Rolling Window      10.687197     10.687197
producto2 Expanding Window       9.677990      9.677990

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Rolling Window
  RMSE Promedio: 10.5461

producto2:
  Mejor método: Walk-Forward
  RMSE Promedio: 9.6780


## Optimización Bayesiana para Encontrar los Mejores Parámetros ARIMA

Utilizamos Optimización Bayesiana (Optuna) para optimizar los parámetros del modelo ARIMA. Aunque tenemos órdenes específicos (ARIMA(0,2,0) para producto1 y ARIMA(2,1,9) para producto2), podemos optimizar otros aspectos o validar que estos órdenes son óptimos ajustando ligeramente los parámetros alrededor de estos valores.


In [13]:
def objective_walk_forward(trial, data, base_order, seasonal_periods=12, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Walk-Forward Validation.
    Optimiza los parámetros alrededor del orden base especificado.
    """
    # Para producto1: base_order = (0, 2, 0), permitir variaciones pequeñas
    # Para producto2: base_order = (2, 1, 9), permitir variaciones pequeñas
    
    p_base, d_base, q_base = base_order
    
    # Permitir pequeñas variaciones alrededor del orden base
    p = trial.suggest_int('p', max(0, p_base - 1), p_base + 1)
    d = trial.suggest_int('d', max(0, d_base - 1), d_base + 1)
    q = trial.suggest_int('q', max(0, q_base - 1), q_base + 2)
    
    order = (p, d, q)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = arima_forecast(
                train_data, 
                order=order,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_rolling_window(trial, data, base_order, train_window_size=24):
    """
    Función objetivo para optimización bayesiana usando Rolling Window Validation.
    """
    p_base, d_base, q_base = base_order
    
    p = trial.suggest_int('p', max(0, p_base - 1), p_base + 1)
    d = trial.suggest_int('d', max(0, d_base - 1), d_base + 1)
    q = trial.suggest_int('q', max(0, q_base - 1), q_base + 2)
    
    order = (p, d, q)
    
    n = len(data)
    errors = []
    
    for i in range(train_window_size, n):
        train_data = data[i - train_window_size:i]
        actual = data[i]
        
        try:
            prediction = arima_forecast(
                train_data, 
                order=order,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_expanding_window(trial, data, base_order, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Expanding Window Validation.
    """
    p_base, d_base, q_base = base_order
    
    p = trial.suggest_int('p', max(0, p_base - 1), p_base + 1)
    d = trial.suggest_int('d', max(0, d_base - 1), d_base + 1)
    q = trial.suggest_int('q', max(0, q_base - 1), q_base + 2)
    
    order = (p, d, q)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = arima_forecast(
                train_data, 
                order=order,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)


In [14]:
# Realizar optimización bayesiana para cada producto usando su mejor método de validación
print("\n" + "=" * 80)
print("OPTIMIZACIÓN BAYESIANA")
print("=" * 80)

mejores_parametros = {}

# Órdenes base para cada producto
ordenes_base = {
    'producto1': ORDER_PRODUCTO1,
    'producto2': ORDER_PRODUCTO2
}

for producto in df.columns:
    print(f"\n--- Optimizando {producto} ---")
    data = df[producto].values
    mejor_metodo = mejor_validacion[producto]
    base_order = ordenes_base[producto]
    
    print(f"{producto}: Mejor método = {mejor_metodo}")
    print(f"{producto}: Orden base = ARIMA{base_order}")
    
    # Seleccionar función objetivo según el mejor método
    if mejor_metodo == 'Walk-Forward':
        objective_func = lambda trial: objective_walk_forward(trial, data, base_order, TRAIN_SIZE_MIN)
    elif mejor_metodo == 'Rolling Window':
        objective_func = lambda trial: objective_rolling_window(trial, data, base_order, TRAIN_WINDOW_SIZE)
    else:  # Expanding Window
        objective_func = lambda trial: objective_expanding_window(trial, data, base_order, TRAIN_SIZE_MIN)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_func, n_trials=30, show_progress_bar=True)
    
    # Obtener mejores parámetros
    best_params = study.best_params
    best_order = (best_params['p'], best_params['d'], best_params['q'])
    best_rmse = study.best_value
    
    mejores_parametros[producto] = {
        'order': best_order,
        'rmse': best_rmse,
        'metodo': mejor_metodo,
        'base_order': base_order
    }
    
    print(f"  Orden base: ARIMA{base_order}")
    print(f"  Mejor orden encontrado: ARIMA{best_order}")
    print(f"  Mejor RMSE: {best_rmse:.4f}")
    print(f"  Método usado: {mejor_metodo}")


[I 2025-12-01 19:41:54,429] A new study created in memory with name: no-name-5b7bfe61-0756-417c-84ae-8dafcfa5b262



OPTIMIZACIÓN BAYESIANA

--- Optimizando producto1 ---
producto1: Mejor método = Rolling Window
producto1: Orden base = ARIMA(0, 2, 0)


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-12-01 19:41:58,701] Trial 0 finished with value: 7.456658292752049 and parameters: {'p': 0, 'd': 1, 'q': 2}. Best is trial 0 with value: 7.456658292752049.
[I 2025-12-01 19:42:00,719] Trial 1 finished with value: 8.17845304068676 and parameters: {'p': 0, 'd': 1, 'q': 1}. Best is trial 0 with value: 7.456658292752049.
[I 2025-12-01 19:42:01,966] Trial 2 finished with value: 20.122966817475586 and parameters: {'p': 0, 'd': 3, 'q': 0}. Best is trial 0 with value: 7.456658292752049.
[I 2025-12-01 19:42:03,206] Trial 3 finished with value: 10.546098291261977 and parameters: {'p': 0, 'd': 2, 'q': 0}. Best is trial 0 with value: 7.456658292752049.
[I 2025-12-01 19:42:04,408] Trial 4 finished with value: 7.929612696116503 and parameters: {'p': 0, 'd': 1, 'q': 0}. Best is trial 0 with value: 7.456658292752049.
[I 2025-12-01 19:42:06,961] Trial 5 finished with value: 6.176604875019757 and parameters: {'p': 1, 'd': 2, 'q': 1}. Best is trial 5 with value: 6.176604875019757.
[I 2025-12-01 1

[I 2025-12-01 19:43:43,820] A new study created in memory with name: no-name-94772712-534a-4994-9f85-18ddf18099aa


[I 2025-12-01 19:43:43,817] Trial 29 finished with value: 6.114134418207321 and parameters: {'p': 1, 'd': 1, 'q': 2}. Best is trial 26 with value: 6.114134418207321.
  Orden base: ARIMA(0, 2, 0)
  Mejor orden encontrado: ARIMA(1, 1, 2)
  Mejor RMSE: 6.1141
  Método usado: Rolling Window

--- Optimizando producto2 ---
producto2: Mejor método = Walk-Forward
producto2: Orden base = ARIMA(2, 1, 9)


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-12-01 19:44:45,759] Trial 0 finished with value: 9.997406104431526 and parameters: {'p': 3, 'd': 1, 'q': 11}. Best is trial 0 with value: 9.997406104431526.
[I 2025-12-01 19:45:13,208] Trial 1 finished with value: 9.231152005091404 and parameters: {'p': 1, 'd': 2, 'q': 9}. Best is trial 1 with value: 9.231152005091404.
[I 2025-12-01 19:45:58,406] Trial 2 finished with value: 10.880061099654588 and parameters: {'p': 2, 'd': 0, 'q': 10}. Best is trial 1 with value: 9.231152005091404.
[I 2025-12-01 19:46:38,730] Trial 3 finished with value: 11.348015912483858 and parameters: {'p': 1, 'd': 0, 'q': 10}. Best is trial 1 with value: 9.231152005091404.
[I 2025-12-01 19:47:10,102] Trial 4 finished with value: 9.15507774281347 and parameters: {'p': 1, 'd': 1, 'q': 9}. Best is trial 4 with value: 9.15507774281347.
[I 2025-12-01 19:47:47,827] Trial 5 finished with value: 9.725808224451342 and parameters: {'p': 2, 'd': 2, 'q': 9}. Best is trial 4 with value: 9.15507774281347.
[I 2025-12-01 

## Validación de Supuestos del Modelo ARIMA

Después del entrenamiento del modelo final, es importante validar que se cumplan los supuestos estadísticos necesarios para que el modelo ARIMA sea válido:

1. **Estacionariedad**: Los residuales deben ser estacionarios
2. **Ruido blanco**: Los residuales deben comportarse como ruido blanco (sin autocorrelación)
3. **No autocorrelación**: No debe haber autocorrelación significativa en los errores
4. **No heterocedasticidad**: La varianza de los residuales debe ser constante
5. **Normalidad**: Los residuales deben seguir una distribución normal


In [15]:
def validar_supuestos_arima(data, order, alpha=0.05):
    """
    Valida los supuestos estadísticos del modelo ARIMA.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    order : tuple
        Orden del modelo ARIMA (p, d, q)
    alpha : float
        Nivel de significancia para las pruebas (por defecto 0.05)
    
    Returns:
    --------
    dict
        Diccionario con los resultados de las validaciones:
        - 'estacionariedad': dict con resultados del test ADF
        - 'ruido_blanco': dict con resultados del test Ljung-Box
        - 'autocorrelacion': dict con resultados del test Ljung-Box
        - 'heterocedasticidad': dict con resultados del test ARCH
        - 'normalidad': dict con resultados del test Jarque-Bera
        - 'residuales': array con los residuales del modelo
        - 'modelo_ajustado': objeto del modelo ajustado
    """
    resultados = {}
    
    # Convertir a pandas Series
    data_series = pd.Series(data)
    
    try:
        # Ajustar el modelo ARIMA
        model = ARIMA(data_series, order=order)
        modelo_ajustado = model.fit()
        residuales = modelo_ajustado.resid.dropna()
        
        resultados['modelo_ajustado'] = modelo_ajustado
        resultados['residuales'] = residuales
        
        # 1. ESTACIONARIEDAD (Test de Dickey-Fuller Aumentado)
        try:
            adf_result = adfuller(residuales, autolag='AIC')
            adf_statistic = adf_result[0]
            adf_pvalue = adf_result[1]
            es_estacionario = adf_pvalue < alpha
            
            resultados['estacionariedad'] = {
                'estadistico': adf_statistic,
                'pvalor': adf_pvalue,
                'se_cumple': es_estacionario,
                'interpretacion': 'Estacionario' if es_estacionario else 'No estacionario'
            }
        except Exception as e:
            resultados['estacionariedad'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
        # 2. RUIDO BLANCO Y AUTOCORRELACIÓN (Test de Ljung-Box)
        try:
            # Usar lags apropiados (máximo 10 o sqrt(n))
            max_lag = min(10, int(np.sqrt(len(residuales))))
            ljung_box = acorr_ljungbox(residuales, lags=max_lag, return_df=True)
            
            # Tomar el p-valor del último lag
            lb_pvalue = ljung_box['lb_pvalue'].iloc[-1]
            lb_statistic = ljung_box['lb_stat'].iloc[-1]
            es_ruido_blanco = lb_pvalue > alpha
            
            resultados['ruido_blanco'] = {
                'estadistico': lb_statistic,
                'pvalor': lb_pvalue,
                'se_cumple': es_ruido_blanco,
                'interpretacion': 'Ruido blanco' if es_ruido_blanco else 'No es ruido blanco'
            }
            
            resultados['autocorrelacion'] = {
                'estadistico': lb_statistic,
                'pvalor': lb_pvalue,
                'se_cumple': es_ruido_blanco,
                'interpretacion': 'Sin autocorrelación' if es_ruido_blanco else 'Autocorrelación presente'
            }
        except Exception as e:
            resultados['ruido_blanco'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
            resultados['autocorrelacion'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
        # 3. HETEROCEDASTICIDAD (Test ARCH)
        try:
            # Test ARCH para heterocedasticidad
            arch_test = het_arch(residuales, maxlag=min(5, len(residuales)//10))
            arch_statistic = arch_test[0]
            arch_pvalue = arch_test[1]
            no_heterocedasticidad = arch_pvalue > alpha
            
            resultados['heterocedasticidad'] = {
                'estadistico': arch_statistic,
                'pvalor': arch_pvalue,
                'se_cumple': no_heterocedasticidad,
                'interpretacion': 'Homocedástico' if no_heterocedasticidad else 'Heterocedástico'
            }
        except Exception as e:
            resultados['heterocedasticidad'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
        # 4. NORMALIDAD (Test de Jarque-Bera)
        try:
            jb_statistic, jb_pvalue = stats.jarque_bera(residuales)
            es_normal = jb_pvalue > alpha
            
            resultados['normalidad'] = {
                'estadistico': jb_statistic,
                'pvalor': jb_pvalue,
                'se_cumple': es_normal,
                'interpretacion': 'Normal' if es_normal else 'No normal'
            }
        except Exception as e:
            resultados['normalidad'] = {
                'estadistico': np.nan,
                'pvalor': np.nan,
                'se_cumple': False,
                'interpretacion': f'Error: {str(e)}'
            }
        
    except Exception as e:
        print(f"Error al ajustar el modelo: {e}")
        resultados['error'] = str(e)
    
    return resultados


## Modelo Final para Producción

Entrenamos el modelo final con todos los datos disponibles usando los mejores parámetros encontrados mediante optimización bayesiana y realizamos la predicción para el siguiente mes (1 pronóstico).


## Validación de Supuestos del Modelo Final

A continuación se validan los supuestos estadísticos del modelo ARIMA final para cada producto.


In [16]:
# Validar supuestos para cada producto
print("\n" + "=" * 80)
print("VALIDACIÓN DE SUPUESTOS DEL MODELO ARIMA")
print("=" * 80)

resultados_validacion = {}
tabla_resumen = []

for producto in df.columns:
    print(f"\n{'='*80}")
    print(f"VALIDACIÓN DE SUPUESTOS - {producto.upper()}")
    print(f"{'='*80}")
    
    data = df[producto].values
    params = mejores_parametros[producto]
    best_order = params['order']
    
    print(f"\nModelo: ARIMA{best_order}")
    
    # Realizar validación de supuestos
    validacion = validar_supuestos_arima(data, best_order, alpha=0.05)
    resultados_validacion[producto] = validacion
    
    # Mostrar resultados detallados
    if 'error' not in validacion:
        print("\n--- Resultados de las Pruebas ---")
        
        # Estacionariedad
        est = validacion['estacionariedad']
        print(f"\n1. ESTACIONARIEDAD (Test ADF):")
        print(f"   Estadístico: {est['estadistico']:.4f}")
        print(f"   p-valor: {est['pvalor']:.4f}")
        print(f"   Resultado: {est['interpretacion']}")
        
        # Ruido blanco
        rb = validacion['ruido_blanco']
        print(f"\n2. RUIDO BLANCO (Test Ljung-Box):")
        print(f"   Estadístico: {rb['estadistico']:.4f}")
        print(f"   p-valor: {rb['pvalor']:.4f}")
        print(f"   Resultado: {rb['interpretacion']}")
        
        # Autocorrelación
        ac = validacion['autocorrelacion']
        print(f"\n3. AUTOCORRELACIÓN (Test Ljung-Box):")
        print(f"   Estadístico: {ac['estadistico']:.4f}")
        print(f"   p-valor: {ac['pvalor']:.4f}")
        print(f"   Resultado: {ac['interpretacion']}")
        
        # Heterocedasticidad
        het = validacion['heterocedasticidad']
        print(f"\n4. HETEROCEDASTICIDAD (Test ARCH):")
        print(f"   Estadístico: {het['estadistico']:.4f}")
        print(f"   p-valor: {het['pvalor']:.4f}")
        print(f"   Resultado: {het['interpretacion']}")
        
        # Normalidad
        norm = validacion['normalidad']
        print(f"\n5. NORMALIDAD (Test Jarque-Bera):")
        print(f"   Estadístico: {norm['estadistico']:.4f}")
        print(f"   p-valor: {norm['pvalor']:.4f}")
        print(f"   Resultado: {norm['interpretacion']}")
        
        # Agregar a tabla resumen
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'Estacionariedad',
            'Se Cumple': 'Sí' if est['se_cumple'] else 'No',
            'p-valor': f"{est['pvalor']:.4f}",
            'Interpretación': est['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'Ruido Blanco',
            'Se Cumple': 'Sí' if rb['se_cumple'] else 'No',
            'p-valor': f"{rb['pvalor']:.4f}",
            'Interpretación': rb['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'No Autocorrelación',
            'Se Cumple': 'Sí' if ac['se_cumple'] else 'No',
            'p-valor': f"{ac['pvalor']:.4f}",
            'Interpretación': ac['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'No Heterocedasticidad',
            'Se Cumple': 'Sí' if het['se_cumple'] else 'No',
            'p-valor': f"{het['pvalor']:.4f}",
            'Interpretación': het['interpretacion']
        })
        tabla_resumen.append({
            'Producto': producto,
            'Supuesto': 'Normalidad',
            'Se Cumple': 'Sí' if norm['se_cumple'] else 'No',
            'p-valor': f"{norm['pvalor']:.4f}",
            'Interpretación': norm['interpretacion']
        })
    else:
        print(f"\nError en la validación: {validacion['error']}")

# Crear DataFrame con tabla resumen
df_resumen_supuestos = pd.DataFrame(tabla_resumen)

# Mostrar tabla resumen
print("\n" + "=" * 80)
print("TABLA RESUMEN DE VALIDACIÓN DE SUPUESTOS")
print("=" * 80)
print(df_resumen_supuestos.to_string(index=False))

# Determinar si se puede continuar con producción
print("\n" + "=" * 80)
print("EVALUACIÓN FINAL - ¿CONTINUAR CON PRODUCCIÓN?")
print("=" * 80)

for producto in df.columns:
    if producto in resultados_validacion and 'error' not in resultados_validacion[producto]:
        validacion = resultados_validacion[producto]
        
        # Contar cuántos supuestos se cumplen
        supuestos_cumplidos = sum([
            validacion['estacionariedad']['se_cumple'],
            validacion['ruido_blanco']['se_cumple'],
            validacion['autocorrelacion']['se_cumple'],
            validacion['heterocedasticidad']['se_cumple'],
            validacion['normalidad']['se_cumple']
        ])
        
        total_supuestos = 5
        porcentaje_cumplimiento = (supuestos_cumplidos / total_supuestos) * 100
        
        # Criterio: al menos 3 de 5 supuestos deben cumplirse
        # Los más críticos son: estacionariedad, ruido blanco y no autocorrelación
        supuestos_criticos = [
            validacion['estacionariedad']['se_cumple'],
            validacion['ruido_blanco']['se_cumple'],
            validacion['autocorrelacion']['se_cumple']
        ]
        criticos_cumplidos = sum(supuestos_criticos)
        
        puede_produccion = criticos_cumplidos >= 2 and supuestos_cumplidos >= 3
        
        print(f"\n{producto}:")
        print(f"  Supuestos cumplidos: {supuestos_cumplidos}/{total_supuestos} ({porcentaje_cumplimiento:.1f}%)")
        print(f"  Supuestos críticos cumplidos: {criticos_cumplidos}/3")
        print(f"  Decisión: {'✓ APROBADO PARA PRODUCCIÓN' if puede_produccion else '✗ NO APROBADO - Revisar modelo'}")
        
        if not puede_produccion:
            print(f"  Recomendación: Revisar el modelo o considerar transformaciones adicionales")
    else:
        print(f"\n{producto}:")
        print(f"  Error en la validación - No se puede evaluar")



VALIDACIÓN DE SUPUESTOS DEL MODELO ARIMA

VALIDACIÓN DE SUPUESTOS - PRODUCTO1

Modelo: ARIMA(1, 1, 2)

--- Resultados de las Pruebas ---

1. ESTACIONARIEDAD (Test ADF):
   Estadístico: -5.1683
   p-valor: 0.0000
   Resultado: Estacionario

2. RUIDO BLANCO (Test Ljung-Box):
   Estadístico: 0.3300
   p-valor: 1.0000
   Resultado: Ruido blanco

3. AUTOCORRELACIÓN (Test Ljung-Box):
   Estadístico: 0.3300
   p-valor: 1.0000
   Resultado: Sin autocorrelación

4. HETEROCEDASTICIDAD (Test ARCH):
   Estadístico: 7.0849
   p-valor: 0.2144
   Resultado: Homocedástico

5. NORMALIDAD (Test Jarque-Bera):
   Estadístico: 72579.8979
   p-valor: 0.0000
   Resultado: No normal

VALIDACIÓN DE SUPUESTOS - PRODUCTO2

Modelo: ARIMA(1, 1, 9)

--- Resultados de las Pruebas ---

1. ESTACIONARIEDAD (Test ADF):
   Estadístico: -22.4993
   p-valor: 0.0000
   Resultado: Estacionario

2. RUIDO BLANCO (Test Ljung-Box):
   Estadístico: 1.0366
   p-valor: 0.9998
   Resultado: Ruido blanco

3. AUTOCORRELACIÓN (Test Lj

In [17]:
# Entrenar modelo final y realizar predicción para producción
print("\n" + "=" * 80)
print("PREDICCIÓN FINAL PARA PRODUCCIÓN")
print("=" * 80)

predicciones_produccion = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    
    # Obtener mejores parámetros de la optimización bayesiana
    params = mejores_parametros[producto]
    best_order = params['order']
    
    print(f"Parámetros del modelo:")
    print(f"  Orden base: ARIMA{params['base_order']}")
    print(f"  Orden optimizado: ARIMA{best_order}")
    print(f"  RMSE en validación: {params['rmse']:.4f}")
    print(f"  Método de validación usado: {params['metodo']}")
    
    # Entrenar modelo final con todos los datos
    try:
        prediccion = arima_forecast(
            data,
            order=best_order,
            forecast_horizon=1
        )
        
        predicciones_produccion[producto] = prediccion
        
        print(f"  Predicción para el siguiente mes: {prediccion:.4f}")
        print(f"  Último valor observado: {data[-1]:.4f}")
        print(f"  Diferencia: {prediccion - data[-1]:.4f}")
        
    except Exception as e:
        print(f"  Error al generar predicción: {e}")
        # Fallback: usar último valor
        predicciones_produccion[producto] = data[-1]
        print(f"  Predicción (fallback): {data[-1]:.4f}")

# Mostrar resumen de predicciones
print("\n" + "=" * 80)
print("RESUMEN DE PREDICCIONES PARA PRODUCCIÓN")
print("=" * 80)
for producto, prediccion in predicciones_produccion.items():
    print(f"{producto}: {prediccion:.4f}")



PREDICCIÓN FINAL PARA PRODUCCIÓN

--- producto1 ---
Parámetros del modelo:
  Orden base: ARIMA(0, 2, 0)
  Orden optimizado: ARIMA(1, 1, 2)
  RMSE en validación: 6.1141
  Método de validación usado: Rolling Window
  Predicción para el siguiente mes: 133.4213
  Último valor observado: 141.9909
  Diferencia: -8.5695

--- producto2 ---
Parámetros del modelo:
  Orden base: ARIMA(2, 1, 9)
  Orden optimizado: ARIMA(1, 1, 9)
  RMSE en validación: 9.1551
  Método de validación usado: Walk-Forward
  Predicción para el siguiente mes: 690.5958
  Último valor observado: 676.0581
  Diferencia: 14.5378

RESUMEN DE PREDICCIONES PARA PRODUCCIÓN
producto1: 133.4213
producto2: 690.5958
